## [한국수자원공사] 가뭄 시설물 코드 정보 openAPI

##### 목적: [한국수자원공사] 가뭄 기상관측소 코드 수집

##### 이유: [한국수자원공사] 가뭄 기상관측 운영정보 openAPI 접근을 위하여, queryString 구성에 관측시설의 시설물 코드가 필요

In [1]:
### 사용할 라이브러리 선언
import pandas as pd
import requests

from bs4 import BeautifulSoup
from urllib.parse import urlencode, quote_plus, unquote

In [2]:
### 1. XML API 요청을 위한 URL 생성 (openAPI 참조문서의 요청명세를 확인하여 필수 queryString 구성 및 인코딩) 
facilityCodeBaseUrl = "http://apis.data.go.kr/B500001/drghtFcltyCode/fcltyList"

serviceKey = "2AlYmSdbDUuKGC0DivNISdLDUqJewAoneYBBTMwCjqcQz8cIiQXSp0je68IfpEVPg6+J2f8OeNy3lak986T1rQ=="
numOfRows = "1000" # 한페이지 결과 수
pageNo = '1' 
fcltyDivCode = "1016" # 시설분류코드
queryString = '?' + urlencode(
    {
        quote_plus('serviceKey') : serviceKey,
        quote_plus('numOfRows') : numOfRows,
        quote_plus('pageNo') : pageNo,
        quote_plus('fcltyDivCode') : fcltyDivCode
    }
)

targetUrl = facilityCodeBaseUrl + queryString

In [3]:
### 2. XML API 요청 및 응답 받기 (BeautifulSoup사용하여 편집)
resp = requests.get(targetUrl) # response 200
resp.encoding="utf-8" # 한글 호환성 확장
xml = resp.text
bsXml = BeautifulSoup(xml, "lxml-xml")

In [4]:
### 3. DataFrame 생성에 필요한 데이터 수집
## 사용할 empty list 
facilityRowList = []
facilityColumnList = []
facilityNameList = []
## 3-1. item 태그들만 추출
rows = bsXml.findAll(name = "item")

In [5]:
## 3-2. 반복문을 통하여 각 행에 들어갈 컬럼값을 추출
for i in range (0, len(rows)):
    columns = rows[i].findAll()
    for j in range (0, len(columns)):
        facilityColumnList.append(columns[j].text)
    facilityRowList.append(facilityColumnList)
    facilityColumnList = [] # 다음 행의 컬럼값을 담기위해 초기화

In [6]:
## 3-3. 컬럼명 추출하기 
columns = rows[0].findAll()
for i in range (0, len(columns)):
    facilityNameList.append(columns[i].name)

In [7]:
## 4. 행기준으로 DataFrame 생성 및 csv로 저장
facilityCodeDf = pd.DataFrame(facilityRowList, columns = facilityNameList)
facilityCodeDf.to_csv(("../dataset/facilityCode_{}.csv".format(fcltyDivCode)), index = False, encoding = "ms949")